# RAG的提示词教程

1. 向量数据库的简单构建
2. RAG的实现

## 1、向量数据库的简单构建

In [23]:
import numpy as np
from typing import List, Tuple

class SimpleVectorDB:
    """简单向量数据库实现"""
    
    def __init__(self):
        """初始化向量数据库"""
        self.vectors = []  # 存储向量
        self.metadata = []  # 存储向量对应的元数据
        self.dim = None  # 向量维度
    
    def add_vector(self, vector: List[float], metadata: dict = None) -> int:
        """
        添加向量到数据库
        
        参数:
            vector: 要添加的向量
            metadata: 向量相关的元数据
            
        返回:
            向量在数据库中的索引
        """
        # 转换为numpy数组
        vector = np.array(vector, dtype=np.float32)
        
        # 检查向量维度
        if self.dim is None:
            self.dim = len(vector)
        elif len(vector) != self.dim:
            raise ValueError(f"向量维度必须为{self.dim}")
        
        # 保存向量和元数据
        self.vectors.append(vector)
        self.metadata.append(metadata or {})
        
        # 返回向量索引
        return len(self.vectors) - 1
    
    def search(self, query_vector: List[float], k: int = 5) -> List[Tuple[int, float, dict]]:
        """
        搜索最相似的向量
        
        参数:
            query_vector: 查询向量
            k: 返回的相似向量数量
            
        返回:
            包含元组的列表，每个元组格式为 (索引, 相似度, 元数据)
        """
        # 转换为numpy数组
        query_vector = np.array(query_vector, dtype=np.float32)
        
        # 计算相似度（使用余弦相似度）
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity, self.metadata[i]))
        
        # 按相似度排序并返回前k个
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:k]
    
    def __len__(self) -> int:
        """返回数据库中向量的数量"""
        return len(self.vectors)

In [ ]:
import numpy as np
from typing import List, Dict, Optional

class VectorStorage:
    """简单向量存储类，仅负责向量的存储功能"""
    
    def __init__(self):
        """初始化向量存储"""
        self.vectors = []  # 存储向量
        self.metadata = []  # 存储向量对应的元数据
        self.dim = None  # 向量维度
    
    def add(self, vector: List[float], metadata: Optional[Dict] = None) -> int:
        """
        添加向量到存储
        
        参数:
            vector: 要添加的向量
            metadata: 向量相关的元数据
            
        返回:
            向量在存储中的索引
        """
        # 转换为numpy数组
        vector = np.array(vector, dtype=np.float32)
        
        # 检查向量维度
        if self.dim is None:
            self.dim = len(vector)
        elif len(vector) != self.dim:
            raise ValueError(f"向量维度必须为{self.dim}")
        
        # 保存向量和元数据
        self.vectors.append(vector)
        self.metadata.append(metadata or {})
        
        # 返回向量索引
        return len(self.vectors) - 1
    
    def get(self, index: int) -> tuple[np.ndarray, dict]:
        """
        获取指定索引的向量和元数据
        
        参数:
            index: 向量索引
            
        返回:
            元组 (向量, 元数据)
        """
        if 0 <= index < len(self.vectors):
            return self.vectors[index], self.metadata[index]
        raise IndexError("索引超出范围")
    def search(self, query: str, k: int = 2) -> List[tuple[int, float, dict]]:
        """        搜索最相似的向量
        参数:
            query_vector: 查询向量
            k: 返回的相似向量数量
        返回:
            包含元组的列表，每个元组格式为 (索引, 相似度, 元数据)
        """
        # 输入的文本转换成向量格式
        query_vector = embedding_model.encode([query], prompt_name="query")

        # 计算相似度
        similarities = embedding_model.similarity(query_vector, self.vectors)

        # 获取最相似的k个向量的索引
        top_k_indices = similarities.argsort()[-k:][::-1]

        # 返回结果
        return [(index, similarities[index], self.metadata[index]) for index in top_k_indices]

    def __len__(self) -> int:
        """返回存储中向量的数量"""
        return len(self.vectors)

In [6]:
# 创建向量数据库实例
db = SimpleVectorDB()

# 添加一些向量
db.add_vector([1, 2, 3], {"id": 1, "name": "向量A"})
db.add_vector([4, 5, 6], {"id": 2, "name": "向量B"})
db.add_vector([7, 8, 9], {"id": 3, "name": "向量C"})

# 搜索相似向量
query = [144, 567, 3.1]
results = db.search(query, k=3)

# 打印搜索结果
print(f"查询向量: {query}")
print("搜索结果:")
for idx, sim, meta in results:
    print(f"索引: {idx}, 相似度: {sim:.4f}, 元数据: {meta}")    

查询向量: [144, 567, 3.1]
搜索结果:
索引: 2, 相似度: 0.6838, 元数据: {'id': 3, 'name': '向量C'}
索引: 1, 相似度: 0.6681, 元数据: {'id': 2, 'name': '向量B'}
索引: 0, 相似度: 0.5881, 元数据: {'id': 1, 'name': '向量A'}


## 2、RAG实现

In [5]:
### 1、文档加载
import PyPDF2

def read_pdf(file_path: str) -> str:
    """
    读取PDF文件内容
    
    参数:
        file_path: PDF文件路径
        
    返回:
        PDF文件中的文本内容
    """
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"读取PDF文件时出错: {e}")
        return ""
    
file_path = "./liulangdiqiu2.pdf"
print(read_pdf(file_path))
print(len(read_pdf(file_path)))

《流浪地球2》是由郭帆执导的科幻灾难电影，于2023年上映，故事围绕《流浪地球》
前作展开，以计划建造1万座行星发动机的时代为背景。
在不远的未来，太阳急速衰老膨胀，即将吞噬太阳系，地球面临灭顶之灾。为应对危机，地
球各国成立联合政府，提出数百个自救计划，其中“移山计划”“方舟计划”“逐月计划”
和“数字生命计划”进入论证阶段。“移山计划”由中国提出，旨在建造1万座行星
发动机推动地球前往新家园；“方舟计划”是美国提议的在地球同步轨道建立空间站以带
领人类逃离；“逐月计划”由俄罗斯提出，想改造月球为逃生舱，后因月球结构等问题并
入“移山计划”；“数字生命计划”则是将人类意识数字化，实现永生，但最终被伦理委
员会禁止。经过考量，“移山计划”被选定，人类开始着手建造行星发动机，同时准备建
造卫星发动机以放逐月球，摆脱月球引力。
然而，计划推进过程中危机四伏。2044年，太空电梯基地遭遇危机，处在9万公里高度
的方舟空间站爆炸坠落，引发连锁反应，导致太空电梯基地被摧毁，流浪地球计划面临重大
挑战。影片中，满腔赤诚的刘培强（吴京饰）历经层层考验成为航天员大队的一员，他与韩
朵朵（王智饰）在此过程中相知相恋，而刘培强也在后续故事中为了地球和家人，不断经历
着艰难抉择与挑战。
另一边，量子科学家图恒宇（刘德华饰）则与“数字生命计划”紧密相关。他致力于将女
儿图丫丫的意识数字化，即使该计划被禁止，他也未放弃。在一系列意外后，图恒宇自己也
意外进入数字世界，以一种特殊的方式继续参与到拯救地球的行动中，他在数字空间中的经
历，也为影片增添了一层神秘的“元宇宙”色彩。
联合政府中国代表周喆直（李雪健饰）则在国际舞台上，为“移山计划”的推进四处奔走，
面对各方压力和危机，他始终坚定信念，相信人类能够团结起来拯救地球，展现出了强大的
责任感与使命感。
随着故事发展，月球发动机建造完成，但却因未知原因被启动，月球开始向地球逼近，引发
了全球性的灾难，地震、海啸等灾害频发，人类再次面临生死考验。为了阻止月球撞击地球，
人类决定启动行星发动机，利用其推力将月球推离。在这一过程中，无数普通人与主角们一
起，克服了重重困难，最终成功放逐月球，为地球开启流浪之旅奠定了基础。
《流浪地球2》通过展现刘培强、图恒宇、周喆直等众多角色的经历，以及全球人类在末
日危机下的挣扎与抗争，呈现了一个宏大而震撼的科幻

In [6]:
### 2、文本分割分块
from typing import List, Tuple, Dict
import re

def split_pdf(text, max_len=200):
    # 按句子优先分割
    chunks = re.split(r'[。！？]', text)
    # 合并短句到200字以内
    result, buf = [], ""
    for s in chunks:
        if len(buf + s) <= max_len:
            buf += s
        else:
            if buf: result.append(buf)
            buf = s
    if buf: result.append(buf)
    return result

chunks = split_pdf(read_pdf(file_path), max_len=200)
chunks

['《流浪地球2》是由郭帆执导的科幻灾难电影，于2023年上映，故事围绕《流浪地球》\n前作展开，以计划建造1万座行星发动机的时代为背景\n在不远的未来，太阳急速衰老膨胀，即将吞噬太阳系，地球面临灭顶之灾为应对危机，地\n球各国成立联合政府，提出数百个自救计划，其中“移山计划”“方舟计划”“逐月计划”\n和“数字生命计划”进入论证阶段',
 '“移山计划”由中国提出，旨在建造1万座行星\n发动机推动地球前往新家园；“方舟计划”是美国提议的在地球同步轨道建立空间站以带\n领人类逃离；“逐月计划”由俄罗斯提出，想改造月球为逃生舱，后因月球结构等问题并\n入“移山计划”；“数字生命计划”则是将人类意识数字化，实现永生，但最终被伦理委\n员会禁止经过考量，“移山计划”被选定，人类开始着手建造行星发动机，同时准备建\n造卫星发动机以放逐月球，摆脱月球引力',
 '\n然而，计划推进过程中危机四伏2044年，太空电梯基地遭遇危机，处在9万公里高度\n的方舟空间站爆炸坠落，引发连锁反应，导致太空电梯基地被摧毁，流浪地球计划面临重大\n挑战影片中，满腔赤诚的刘培强（吴京饰）历经层层考验成为航天员大队的一员，他与韩\n朵朵（王智饰）在此过程中相知相恋，而刘培强也在后续故事中为了地球和家人，不断经历\n着艰难抉择与挑战',
 '\n另一边，量子科学家图恒宇（刘德华饰）则与“数字生命计划”紧密相关他致力于将女\n儿图丫丫的意识数字化，即使该计划被禁止，他也未放弃在一系列意外后，图恒宇自己也\n意外进入数字世界，以一种特殊的方式继续参与到拯救地球的行动中，他在数字空间中的经\n历，也为影片增添了一层神秘的“元宇宙”色彩',
 '\n联合政府中国代表周喆直（李雪健饰）则在国际舞台上，为“移山计划”的推进四处奔走，\n面对各方压力和危机，他始终坚定信念，相信人类能够团结起来拯救地球，展现出了强大的\n责任感与使命感\n随着故事发展，月球发动机建造完成，但却因未知原因被启动，月球开始向地球逼近，引发\n了全球性的灾难，地震、海啸等灾害频发，人类再次面临生死考验为了阻止月球撞击地球，\n人类决定启动行星发动机，利用其推力将月球推离',
 '在这一过程中，无数普通人与主角们一\n起，克服了重重困难，最终成功放逐月球，为地球开启流浪之旅奠定了基础',
 '\n《流浪地球2》通过展现刘培强、图恒宇、周喆

In [1]:
### 3、embedding
from sentence_transformers import SentenceTransformer

embedding_model_path = "/home/lixinyu/weights/Qwen3-Embedding-0.6B"
embedding_model = SentenceTransformer(embedding_model_path)

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
queries = [
    "What is the capital of China?",
    "The capital of China is Beijing.",

]
documents = [
  "The capital of China is Beijing.",
  "What is the capital of China?",
  ]

query_embeddings = embedding_model.encode(queries, prompt_name="query")
document_embeddings = embedding_model.encode(documents)
print(query_embeddings.shape)
similarity = embedding_model.similarity(query_embeddings, document_embeddings)
print(similarity)

(2, 1024)
tensor([[0.7646, 0.7851],
        [0.8339, 0.7323]])


In [22]:
embedding_pdf = embedding_model.encode(chunks, prompt_name="document")

example1=["“数字生命计划”由谁提出？"]
example2=embedding_pdf
print(example2.shape)
example1 = embedding_model.encode(example1, prompt_name="query")
similarity = embedding_model.similarity(example1, example2)
print(similarity)

print(embedding_pdf)

(7, 1024)
tensor([[0.3049, 0.5723, 0.2594, 0.5323, 0.3495, 0.2657, 0.1746]])
[[ 0.00891668 -0.01762075 -0.01036646 ... -0.08403418  0.01742308
   0.02130047]
 [ 0.02119874 -0.01577233 -0.01156385 ... -0.01115059  0.00554845
   0.01665588]
 [ 0.09470636  0.01157483 -0.01174384 ... -0.02367868  0.02360488
   0.0048494 ]
 ...
 [ 0.05936852  0.00221355 -0.01172859 ... -0.02255914  0.01159022
  -0.01121389]
 [ 0.03398024 -0.01341954 -0.0115754  ... -0.01548578 -0.04122541
  -0.0325289 ]
 [ 0.03194692 -0.03096953 -0.00551068 ... -0.05707911  0.00820924
   0.02514409]]


In [25]:
### 4、存入向量数据库
vb= VectorStorage()
for i,emb in enumerate(embedding_pdf):
    vb.add(emb, {"index": i, "chunk": chunks[i]})

# 获取向量
vector1, meta1 = vb.get(1)
print(f"向量1: {vector1}, 元数据: {meta1}")

# 获取向量数量
print(f"存储的向量数量: {len(vb)}")

向量1: [ 0.02119874 -0.01577233 -0.01156385 ... -0.01115059  0.00554845
  0.01665588], 元数据: {'index': 1, 'chunk': '“移山计划”由中国提出，旨在建造1万座行星\n发动机推动地球前往新家园；“方舟计划”是美国提议的在地球同步轨道建立空间站以带\n领人类逃离；“逐月计划”由俄罗斯提出，想改造月球为逃生舱，后因月球结构等问题并\n入“移山计划”；“数字生命计划”则是将人类意识数字化，实现永生，但最终被伦理委\n员会禁止经过考量，“移山计划”被选定，人类开始着手建造行星发动机，同时准备建\n造卫星发动机以放逐月球，摆脱月球引力'}
存储的向量数量: 7


In [29]:
### 5、检索向量数据库
query = "“数字生命计划”的内容是什么？"
query_vector = embedding_model.encode([query], prompt_name="query")

similarity = embedding_model.similarity(query_vector, embedding_pdf)
print(similarity)

tensor([[0.2921, 0.5227, 0.2679, 0.5337, 0.3160, 0.2771, 0.1876]])


# 整理代码

In [1]:
### 1、加载embedding模型
from sentence_transformers import SentenceTransformer

embedding_model_path = "/home/lixinyu/weights/Qwen3-Embedding-0.6B"
embedding_model = SentenceTransformer(embedding_model_path)

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
### 2、构建向量数据库
import numpy as np
from typing import List, Dict, Optional

class VectorStorage:
    """简单向量存储类，仅负责向量的存储功能"""
    
    def __init__(self):
        """初始化向量存储"""
        self.vectors = []  # 存储向量
        self.metadata = []  # 存储向量对应的元数据
        self.dim = None  # 向量维度
    
    def add(self, vector: List[float], metadata: Optional[Dict] = None) -> int:
        """
        添加向量到存储
        
        参数:
            vector: 要添加的向量
            metadata: 向量相关的元数据
            
        返回:
            向量在存储中的索引
        """
        
        # 检查向量维度
        if self.dim is None:
            self.dim = len(vector)
        elif len(vector) != self.dim:
            raise ValueError(f"向量维度必须为{self.dim}")
        
        # 保存向量和元数据
        self.vectors.append(vector)
        self.metadata.append(metadata or {})
        
        # 返回向量索引
        return len(self.vectors) - 1
    
    def get(self, index: int) -> tuple[np.ndarray, dict]:
        """
        获取指定索引的向量和元数据
        
        参数:
            index: 向量索引
            
        返回:
            元组 (向量, 元数据)
        """
        if 0 <= index < len(self.vectors):
            return self.vectors[index], self.metadata[index]
        raise IndexError("索引超出范围")
    def search(self, query: str, k: int = 2) -> List[tuple[int, float, dict]]:
        """        搜索最相似的向量
        参数:
            query_vector: 查询向量
            k: 返回的相似向量数量
        返回:
            包含元组的列表，每个元组格式为 (索引, 相似度, 元数据)
        """
        # 输入的文本转换成向量格式
        query_vector = embedding_model.encode([query], prompt_name="query")

        # 计算相似度
        similarities = embedding_model.similarity(query_vector, self.vectors)
        similarities = similarities.squeeze()
        
        # 限制 k 值不超过向量总数
        k = min(k, len(self.vectors))
        
        # 获取最相似的 k 个向量的索引（使用 PyTorch 方法）
        top_k_values, top_k_indices = similarities.topk(k, largest=True)
        
        # 返回结果（将张量转换为 Python 列表）
        return [
        (top_k_indices[i].item(), top_k_values[i].item(), self.metadata[top_k_indices[i].item()])
        for i in range(k)
        ]

    def __len__(self) -> int:
        """返回存储中向量的数量"""
        return len(self.vectors)

In [4]:
### 3、文档加载和分块
import PyPDF2
from typing import List, Tuple, Dict
import re

def read_pdf(file_path: str) -> str:
    """
    读取PDF文件内容
    
    参数:
        file_path: PDF文件路径
        
    返回:
        PDF文件中的文本内容
    """
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"读取PDF文件时出错: {e}")
        return ""

def split_pdf(text, max_len=200):
    # 按句子优先分割
    chunks = re.split(r'[。！？]', text)
    # 合并短句到200字以内
    result, buf = [], ""
    for s in chunks:
        if len(buf + s) <= max_len:
            buf += s
        else:
            if buf: result.append(buf)
            buf = s
    if buf: result.append(buf)
    return result

file_path = "./liulangdiqiu2.pdf"
chunks = split_pdf(read_pdf(file_path), max_len=200)
chunks

['《流浪地球2》是由郭帆执导的科幻灾难电影，于2023年上映，故事围绕《流浪地球》\n前作展开，以计划建造1万座行星发动机的时代为背景\n在不远的未来，太阳急速衰老膨胀，即将吞噬太阳系，地球面临灭顶之灾为应对危机，地\n球各国成立联合政府，提出数百个自救计划，其中“移山计划”“方舟计划”“逐月计划”\n和“数字生命计划”进入论证阶段',
 '“移山计划”由中国提出，旨在建造1万座行星\n发动机推动地球前往新家园；“方舟计划”是美国提议的在地球同步轨道建立空间站以带\n领人类逃离；“逐月计划”由俄罗斯提出，想改造月球为逃生舱，后因月球结构等问题并\n入“移山计划”；“数字生命计划”则是将人类意识数字化，实现永生，但最终被伦理委\n员会禁止经过考量，“移山计划”被选定，人类开始着手建造行星发动机，同时准备建\n造卫星发动机以放逐月球，摆脱月球引力',
 '\n然而，计划推进过程中危机四伏2044年，太空电梯基地遭遇危机，处在9万公里高度\n的方舟空间站爆炸坠落，引发连锁反应，导致太空电梯基地被摧毁，流浪地球计划面临重大\n挑战影片中，满腔赤诚的刘培强（吴京饰）历经层层考验成为航天员大队的一员，他与韩\n朵朵（王智饰）在此过程中相知相恋，而刘培强也在后续故事中为了地球和家人，不断经历\n着艰难抉择与挑战',
 '\n另一边，量子科学家图恒宇（刘德华饰）则与“数字生命计划”紧密相关他致力于将女\n儿图丫丫的意识数字化，即使该计划被禁止，他也未放弃在一系列意外后，图恒宇自己也\n意外进入数字世界，以一种特殊的方式继续参与到拯救地球的行动中，他在数字空间中的经\n历，也为影片增添了一层神秘的“元宇宙”色彩',
 '\n联合政府中国代表周喆直（李雪健饰）则在国际舞台上，为“移山计划”的推进四处奔走，\n面对各方压力和危机，他始终坚定信念，相信人类能够团结起来拯救地球，展现出了强大的\n责任感与使命感\n随着故事发展，月球发动机建造完成，但却因未知原因被启动，月球开始向地球逼近，引发\n了全球性的灾难，地震、海啸等灾害频发，人类再次面临生死考验为了阻止月球撞击地球，\n人类决定启动行星发动机，利用其推力将月球推离',
 '在这一过程中，无数普通人与主角们一\n起，克服了重重困难，最终成功放逐月球，为地球开启流浪之旅奠定了基础',
 '\n《流浪地球2》通过展现刘培强、图恒宇、周喆

In [ ]:
### 4、载入向量数据库
vb = VectorStorage()

embedding_pdf = embedding_model.encode(chunks, prompt_name="document")

for i,emb in enumerate(embedding_pdf):
    vb.add(emb, {"index": i, "chunk": chunks[i]})

# 获取向量数量
print(f"存储的向量数量: {len(vb)}")


存储的向量数量: 7
[ 0.00891668 -0.01762075 -0.01036646 ... -0.08403418  0.01742308
  0.02130047]


In [37]:
import torch
similarities = [[0.2921, 0.5227, 0.2679, 0.5337, 0.3160, 0.2771, 0.1876]]
similarities = torch.tensor(similarities)
a,b=similarities.topk(3, largest=True)
print(a)
print(b)

tensor([[0.5337, 0.5227, 0.3160]])
tensor([[3, 1, 4]])


In [66]:
### 5、检索向量数据库
query="《黑客帝国》是由谁导演的？"

results = vb.search(query, k=2)
print(results)


[(0, 0.3406886160373688, {'index': 0, 'chunk': '《流浪地球2》是由郭帆执导的科幻灾难电影，于2023年上映，故事围绕《流浪地球》\n前作展开，以计划建造1万座行星发动机的时代为背景\n在不远的未来，太阳急速衰老膨胀，即将吞噬太阳系，地球面临灭顶之灾为应对危机，地\n球各国成立联合政府，提出数百个自救计划，其中“移山计划”“方舟计划”“逐月计划”\n和“数字生命计划”进入论证阶段'}), (3, 0.31884729862213135, {'index': 3, 'chunk': '\n另一边，量子科学家图恒宇（刘德华饰）则与“数字生命计划”紧密相关他致力于将女\n儿图丫丫的意识数字化，即使该计划被禁止，他也未放弃在一系列意外后，图恒宇自己也\n意外进入数字世界，以一种特殊的方式继续参与到拯救地球的行动中，他在数字空间中的经\n历，也为影片增添了一层神秘的“元宇宙”色彩'})]


In [67]:
### 6、提示词增强
query=query
context = "\n".join([f"{i+1}. {result[2]['chunk']}" for i, result in enumerate(results)])

prompt=f"""
你是一个非常专业的AI智能助手，请你根据下面的“问题”，结合给出的“文本”内容，生成合理的回答，如果文本中没有相关内容，请回答“无法回答”。
问题: {query}
文本: {context}
回答:
"""

print(prompt.format(query=query, context=context))


你是一个非常专业的AI智能助手，请你根据下面的“问题”，结合给出的“文本”内容，生成合理的回答，如果文本中没有相关内容，请回答“无法回答”。
问题: 《黑客帝国》是由谁导演的？
文本: 1. 《流浪地球2》是由郭帆执导的科幻灾难电影，于2023年上映，故事围绕《流浪地球》
前作展开，以计划建造1万座行星发动机的时代为背景
在不远的未来，太阳急速衰老膨胀，即将吞噬太阳系，地球面临灭顶之灾为应对危机，地
球各国成立联合政府，提出数百个自救计划，其中“移山计划”“方舟计划”“逐月计划”
和“数字生命计划”进入论证阶段
2. 
另一边，量子科学家图恒宇（刘德华饰）则与“数字生命计划”紧密相关他致力于将女
儿图丫丫的意识数字化，即使该计划被禁止，他也未放弃在一系列意外后，图恒宇自己也
意外进入数字世界，以一种特殊的方式继续参与到拯救地球的行动中，他在数字空间中的经
历，也为影片增添了一层神秘的“元宇宙”色彩
回答:



In [56]:
### 7、执行提示词
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='cuda:1', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]


In [65]:
### 推理代码
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

《流浪地球2》是由郭帆执导的。
